# Fleet Management - Live Demo (with TRT)

In this notebook we'll show how you can follow an object with JetBot!  We'll use a pre-trained neural network
that was trained on the [COCO dataset](http://cocodataset.org) to detect 90 different common objects.  These include

* Person (index 0)
* Cup (index 47)

and many others (you can check [this file](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt) for a full list of class indices).  The model is sourced from the [TensorFlow object detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)
which provides utilities for training object detectors for custom tasks also!  Once the model is trained, we optimize it using NVIDIA TensorRT on the Jetson Nano.

This makes the network very fast, capable of real-time execution on Jetson Nano!  We won't run through all of the training and optimization steps in this notebook though.

Anyways, let's get started.  First, we'll want to import the ``Object_Follower`` class which takes our pre-trained SSD engine or yolo engine.

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets
from ipywidgets.widgets import Box, HBox, VBox, Layout, Label, Output
from IPython.display import clear_output
import traitlets


In [ ]:
# %pip install pandas
from jetbot.utils import model_selection
from jetbot import FleeterTRT

In [ ]:

FL = FleeterTRT(init_sensor_fm=True)
FL.conf_th = 0.5

od_trt_ms = model_selection(core_library = "TensorRT")
od_trt_ms.model_function = "object detection"

od_model_type_widget = widgets.Select(options=od_trt_ms.model_type_list, value=od_trt_ms.model_type_list[0], description='Object Detection Model Type:')
traitlets.dlink((od_trt_ms, 'model_type_list'), (od_model_type_widget, 'options'))
traitlets.dlink((od_model_type_widget, 'value'), (od_trt_ms, 'model_type'))
traitlets.dlink((od_trt_ms, 'model_type'), (FL, 'type_follower_model'))

od_model_path_widget = widgets.Select(options=od_trt_ms.model_path_list, description='Model Path:', layout=Layout(width='60%'))
traitlets.dlink((od_trt_ms, 'model_path_list'), (od_model_path_widget, 'options'))
traitlets.dlink((od_model_path_widget, 'value'), (od_trt_ms, 'model_path'))
traitlets.dlink((od_trt_ms, 'model_path'), (FL, 'follower_model'))


In [ ]:
rd_trt_ms = model_selection(core_library = "TensorRT")
rd_trt_ms.model_function = "classifier"

rd_model_type_widget = widgets.Select(options=rd_trt_ms.model_type_list, value=rd_trt_ms.model_type_list[0], description='Classifier Model Type:')
traitlets.dlink((rd_trt_ms, 'model_type_list'), (rd_model_type_widget, 'options'))
traitlets.dlink((rd_model_type_widget, 'value'), (rd_trt_ms, 'model_type'))
traitlets.dlink((rd_trt_ms, 'model_type'), (FL, 'type_cruiser_model'))

rd_model_path_widget = widgets.Select(options=rd_trt_ms.model_path_list, description='Model Path:', layout=Layout(width='60%'))
traitlets.dlink((rd_trt_ms, 'model_path_list'), (rd_model_path_widget, 'options'))
traitlets.dlink((rd_model_path_widget, 'value'), (rd_trt_ms, 'model_path'))
traitlets.dlink((rd_trt_ms, 'model_path'), (FL, 'cruiser_model'))
traitlets.dlink((rd_trt_ms, 'preprocess_path'), (FL, 'cruiser_model_preprocess'))


The followings is to construct the control widgets for fleet control.

In [ ]:

# image_widget = widgets.Image(format='jpeg', width=OF.img_width, height=OF.img_height)
image_widget = widgets.Image(format='jpeg', width=300, height=300, layout = Layout(align_self='center'))

# display(image_widget)
traitlets.dlink((FL, 'cap_image'), (image_widget, 'value'))

# display buttons for start and stop running
button_layout = widgets.Layout(width='150px', height='40px', align_self='center')
stop_button = widgets.Button(description='Stop', tooltip='Click to stop running', icon='stop', layout=button_layout)
stop_button.style.button_color='Red'

start_button = widgets.Button(description='Start', tooltip='Click to start running', icon='play', layout=button_layout)
start_button.style.button_color='lightBlue'

button_box = widgets.HBox([start_button, stop_button], layout=widgets.Layout(justify_content='space-around', width='30%'))


In [ ]:
# infos and params of fleeting control
w_layout = Layout(flex='1 1 auto', width='auto')
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked', layout=w_layout)
object_view_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='object view', layout=w_layout)
label_widget = widgets.BoundedIntText(value=1, min=0, max=100, step=1, description='tracked label', layout=w_layout)  # target to be tracked
label_text_widget = widgets.Text(value='', description='label name', layout=w_layout)  # target name to be tracked
speed_widget = widgets.FloatSlider(min=0, max=1.0, description='speed', readout_format='.3f', layout=w_layout)
speed_gain_widget = widgets.FloatSlider(value=0.01, min=0.001, max=0.2, step=0.001, description='speed_gain', readout_format='.3f', layout=w_layout)
speed_dev_widget = widgets.FloatSlider(value=0.4, min=0.01, max=1.0, step=0.01, description='speed_dev', readout_format='.3f', layout=w_layout)
turn_gain_widget = widgets.FloatSlider(value=0.25, min=0.05, max=0.5, step=0.001, description='turn gain', readout_format='.3f', layout=w_layout)
steering_bias_widget = widgets.FloatSlider(value=0.0, min=-0.1, max=0.1, step=0.001, description='steering bias', readout_format='.3f', layout=w_layout)
view_target_widget = widgets.FloatSlider(value=0.2, min=0.001, max=1.0, step=0.001, description='view_target', readout_format='.3f', layout=w_layout)

traitlets.dlink((FL, 'blocked'), (blocked_widget, 'value'))
traitlets.dlink((FL, 'mean_view'), (object_view_widget, 'value'))
traitlets.dlink((label_widget, 'value'), (FL, 'label'))
traitlets.dlink((FL, 'label_text'), (label_text_widget, 'value'))
traitlets.dlink((turn_gain_widget, 'value'), (FL, 'turn_gain_fm'))
traitlets.dlink((speed_gain_widget, 'value'), (FL, 'speed_gain_fm'))
traitlets.dlink((speed_dev_widget, 'value'), (FL, 'speed_dev_fm'))
traitlets.dlink((FL, 'speed_fm'), (speed_widget, 'value'))
traitlets.dlink((steering_bias_widget, 'value'), (FL, 'steering_bias_fm'))
traitlets.dlink((view_target_widget, 'value'), (FL, 'target_view'))

In [ ]:
# control params of road cruising control
speed_gain_slider = widgets.FloatSlider(min=0, max=1, step=0.001, value=0.3, description='speed gain', readout_format='.3f')
steering_gain_slider = widgets.FloatSlider(min=0, max=0.5, step=0.001, value=0.2, description='steering gain', readout_format='.3f')
steering_dgain_slider = widgets.FloatSlider(min=0, max=2.0, step=0.001, value=1.2, description='steering kd', readout_format='.3f')
steering_bias_slider = widgets.FloatSlider(min=-0.1, max=0.1, step=0.001, value=-0.01, description='steering bias', readout_format='.3f')

traitlets.dlink((speed_gain_slider, 'value'), (FL, 'speed_gain_rc'))
traitlets.dlink((steering_gain_slider, 'value'), (FL, 'steering_gain_rc'))
traitlets.dlink((steering_dgain_slider, 'value'), (FL, 'steering_dgain_rc'))
traitlets.dlink((steering_bias_slider, 'value'), (FL, 'steering_bias_rc'))

VBox_control = VBox([speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider], layout=Layout(align_self='center'))

# stat information of road cruising
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = widgets.FloatSlider(min=0, max=2.0, orientation='vertical', description='y')
steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = widgets.FloatSlider(min=0.0, max=1.0, orientation='vertical', description='speed')

traitlets.dlink((FL, 'x_slider'), (x_slider, 'value'))
traitlets.dlink((FL, 'y_slider'), (y_slider, 'value'))
traitlets.dlink((FL, 'steering_rc'), (steering_slider, 'value'))
traitlets.dlink((FL, 'speed_rc'), (speed_slider, 'value'))

Box_y_state = HBox([y_slider, speed_slider])
Box_x_state = VBox([x_slider, steering_slider])
Box_state = VBox([Box_y_state, Box_x_state])


In [ ]:
out = Output(layout={'border': '2px solid black'})

@out.capture()
def start(change):
    FL.start_fm(change)
    
@out.capture()
def stop(change):
    FL.stop_fm(change)
    %reset -f

In [ ]:
RC_box_layout = Layout(display='flex', flex_flow='column', align_items='stretch', border='solid 2px', width='60%')
FL_box_layout = Layout(display='flex', flex_flow='column', align_items='stretch', border='solid 2px', width='60%')
H_box_layout = Layout(display='flex', flex_flow='row', align_items='stretch')
label_layout = Layout(display='flex', flex_flow='row', align_items='stretch', align_self='center')

# Fleet control widgets layout
Fleet_Control_items = [HBox([Label(value='--- Fleet Control ---')], layout=Layout(align_self='center')),
                       HBox([blocked_widget, object_view_widget], layout = H_box_layout),
                       HBox([speed_widget, view_target_widget], layout = H_box_layout),
                       HBox([speed_gain_widget, speed_dev_widget], layout = H_box_layout),
                       HBox([turn_gain_widget, steering_bias_widget], layout = H_box_layout)]

# Road cruising control widgets layout
RC_control_items = [HBox([Label(value='--- Cruising Control ---')], layout=Layout(align_self='center')),
                    HBox([Box_state, VBox_control], layout = H_box_layout)]

image_items = [image_widget, VBox([label_text_widget, label_widget])]

display(HBox([od_model_type_widget, od_model_path_widget], layout={'border': '2px solid black'}))
display(HBox([rd_model_type_widget, rd_model_path_widget], layout={'border': '2px solid black'}))

display(HBox([VBox(children = image_items, layout = Layout(align_self='center')),
              Box(children = RC_control_items, layout = RC_box_layout)], layout = Layout(justify_content='space-between')),
        HBox([button_box, Box(children = Fleet_Control_items, layout = FL_box_layout)], layout = Layout(align_items='stretch', justify_content='space-between')))

start_button.on_click(start)
stop_button.on_click(stop)

out